# Covid Around The World

![Covid](../images/covid.jpeg)

## Install Widgets
ipwidgets is a library that allows you to interact with your data.

In [24]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Import Data Analysis Libraries
These are Python libraries specifically designed to manuplate  eg import, join datasets, analyise, filter, and clean data.

In [25]:
import pandas as pd
import numpy as np

## Get Source Data
Data is sourced from a git hub repository for Johns Hopkins University COVID Data. The information is updated daily. The link is here:https://github.com/CSSEGISandData. <br>
<b>Note</b> Data is updated daily.


In [26]:
#Recorded deaths since 1/1/2020
death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
#Confirmed Cases since 1/1/2020
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
#Recoveries since C since 1/1/2020
recovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
#Summanation of data by Country
country_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv", index_col=[0])

In [27]:
# Check country data (Just type in the name of the dataframe and how many rows from the top(head))
country_df.head(3)

,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
Country_Region,,,,,,,,,,,,,
Afghanistan,2021-02-25 19:26:30,33.93911,67.709953,55680.0,2438.0,49281.0,3961.0,143.032039,NaN,NaN,4.378592,4,AFG
Albania,2021-02-25 19:26:30,41.15330,20.168300,104313.0,1736.0,67158.0,35419.0,3624.748071,NaN,NaN,1.664222,8,ALB
Algeria,2021-02-25 19:26:30,28.03390,1.659600,112461.0,2970.0,77537.0,31954.0,256.461403,NaN,NaN,2.640916,12,DZA


In [28]:
# Check data has been imported correctly
confirmed_df.head(3)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,55514,55518,55540,55557,55575,55580,55604,55617,55646,55664
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,93850,94651,95726,96838,97909,99062,100246,101285,102306,103327
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,110894,111069,111247,111418,111600,111764,111917,112094,112279,112461


In [29]:
#Get the sape of the data
print (confirmed_df.shape)
print (recovered_df.shape)
print (death_df.shape)

(274, 404)
(259, 404)
(274, 404)


## Data Cleansing
Data Cleansing is an important step in analysing data:
Here we change the column headings to lower case and simplify column heading names, and eliminate missing data. (Note normally you would not eliminate missing data without carrying out a missing data analysis)

In [30]:
# Convert to Lowercase
#country_df
country_df.columns = map(str.lower,country_df.columns)
#confirmed_df
confirmed_df.columns = map(str.lower,confirmed_df.columns)
#recovered_df
recovered_df.columns = map(str.lower,recovered_df.columns)
#death_df
death_df.columns = map(str.lower,death_df.columns)


In [31]:
# Check 
# country_df.head(2)
#confirmed_df.head(2)
recovered_df.head(2)
# death_df.head(2)

,province/state,country/region,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,48433,48514,48626,48798,48803,48820,48834,48895,48967,49086
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,57736,58794,59684,60675,61605,62533,63329,64318,65403,66309


In [32]:
# Rename columns to make it easier to refer to them
country_df = country_df.rename(columns={"country_region":"country","long_":"long"})
confirmed_df =confirmed_df.rename(columns = {'province/state':'state','country/region':'country'} )
recovered_df = recovered_df.rename(columns = {'province/state':'state','country/region':'country' } )
death_df =death_df.rename(columns = {'province/state':'state','country/region':'country' } )

In [33]:
# displaying the datatypes and converting to integers
#display(country_df.dtypes) 
  
# converting 'Field_2' and 'Field_3' from float to int 
country_df['confirmed'] = country_df['confirmed'].apply(np.int64) 
country_df['deaths'] = country_df['deaths'].apply(np.int64) 
country_df['recovered'] = country_df['confirmed'].apply(np.int64) 
country_df['active'] = country_df['deaths'].apply(np.int64)   
# displaying the datatypes 
#display(country_df.dtypes) 

In [34]:
# Recheck
country_df.head(2)
# confirmed_df.head(2)
# recovered_df.head(2)
# death_df.head(2)

,last_update,lat,long,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
Country_Region,,,,,,,,,,,,,
Afghanistan,2021-02-25 19:26:30,33.93911,67.709953,55680,2438,55680,2438,143.032039,NaN,NaN,4.378592,4,AFG
Albania,2021-02-25 19:26:30,41.15330,20.168300,104313,1736,104313,1736,3624.748071,NaN,NaN,1.664222,8,ALB


## Sort Country Data 
Data is sorted in order of the number of confirmed cases. <br>
**Note** we are creating a new dataframe.

In [35]:
sorted_country_df = country_df.sort_values('confirmed', ascending = False)

In [36]:
sorted_country_df.head(10)

,last_update,lat,long,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
Country_Region,,,,,,,,,,,,,
US,2021-02-25 19:26:30,40.000000,-100.000000,28383517,507146,28383517,507146,8614.999004,NaN,NaN,1.786762,840,USA
India,2021-02-25 19:26:30,20.593684,78.962880,11046914,156705,11046914,156705,800.498471,NaN,NaN,1.418541,356,IND
Brazil,2021-02-25 19:26:30,-14.235000,-51.925300,10324463,249957,10324463,249957,4857.212884,NaN,NaN,2.421017,76,BRA
United Kingdom,2021-02-25 19:26:30,55.000000,-3.000000,4166718,122303,4166718,122303,6137.815978,NaN,NaN,2.935236,826,GBR
Russia,2021-02-25 19:26:30,61.524000,105.318800,4164802,83481,4164802,83481,2853.885230,NaN,NaN,2.004441,643,RUS
France,2021-02-25 19:26:30,46.227600,2.213700,3746466,85734,3746466,85734,5739.642138,NaN,NaN,2.288397,250,FRA
Spain,2021-02-25 19:26:30,40.463667,-3.749220,3180212,68813,3180212,68813,6801.896610,NaN,NaN,2.163787,724,ESP
Italy,2021-02-25 19:26:30,41.871900,12.567400,2868435,96974,2868435,96974,4744.208197,NaN,NaN,3.380729,380,ITA
Turkey,2021-02-25 19:26:30,38.963700,35.243300,2674766,28358,2674766,28358,3171.443668,NaN,NaN,1.060205,792,TUR


In [37]:
# Create a function that will colour some of the columns for emphasis
def highlight_col(x):
    r = 'background-color: #7ED5EA'
    p = 'background-color: #63BCE5'
    g = 'background-color: #4B9FE1'
    temp_df = pd.DataFrame ('',index = x.index, columns = x.columns)
    temp_df.iloc[:,4]  = p
    temp_df.iloc[:,5]  = r
    temp_df.iloc[:,6]  = g
    return temp_df

In [38]:
# Print the first 5 and style (calling the function)
sorted_country_df.head(5).style.apply(highlight_col, axis = None)

,last_update,lat,long,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
Country_Region,,,,,,,,,,,,,
US,2021-02-25 19:26:30,40.000000,-100.000000,28383517,507146,28383517,507146,8614.999004,nan,nan,1.786762,840,USA
India,2021-02-25 19:26:30,20.593684,78.962880,11046914,156705,11046914,156705,800.498471,nan,nan,1.418541,356,IND
Brazil,2021-02-25 19:26:30,-14.235000,-51.925300,10324463,249957,10324463,249957,4857.212884,nan,nan,2.421017,76,BRA
United Kingdom,2021-02-25 19:26:30,55.000000,-3.000000,4166718,122303,4166718,122303,6137.815978,nan,nan,2.935236,826,GBR
Russia,2021-02-25 19:26:30,61.524000,105.318800,4164802,83481,4164802,83481,2853.885230,nan,nan,2.004441,643,RUS


In [39]:

sorted_country_df.style.format({
    'confirmed': '{:,.2f}'.format,
    'deaths': '{:,.2f}'.format,
    'recovered': '{:,.2%}'.format,
})
#sorted_country_df.head(3)

,last_update,lat,long,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
Country_Region,,,,,,,,,,,,,
US,2021-02-25 19:26:30,40.000000,-100.000000,"28,383,517.00","507,146.00","2,838,351,700.00%",507146,8614.999004,nan,nan,1.786762,840,USA
India,2021-02-25 19:26:30,20.593684,78.962880,"11,046,914.00","156,705.00","1,104,691,400.00%",156705,800.498471,nan,nan,1.418541,356,IND
Brazil,2021-02-25 19:26:30,-14.235000,-51.925300,"10,324,463.00","249,957.00","1,032,446,300.00%",249957,4857.212884,nan,nan,2.421017,76,BRA
United Kingdom,2021-02-25 19:26:30,55.000000,-3.000000,"4,166,718.00","122,303.00","416,671,800.00%",122303,6137.815978,nan,nan,2.935236,826,GBR
Russia,2021-02-25 19:26:30,61.524000,105.318800,"4,164,802.00","83,481.00","416,480,200.00%",83481,2853.885230,nan,nan,2.004441,643,RUS
France,2021-02-25 19:26:30,46.227600,2.213700,"3,746,466.00","85,734.00","374,646,600.00%",85734,5739.642138,nan,nan,2.288397,250,FRA
Spain,2021-02-25 19:26:30,40.463667,-3.749220,"3,180,212.00","68,813.00","318,021,200.00%",68813,6801.896610,nan,nan,2.163787,724,ESP
Italy,2021-02-25 19:26:30,41.871900,12.567400,"2,868,435.00","96,974.00","286,843,500.00%",96974,4744.208197,nan,nan,3.380729,380,ITA
Turkey,2021-02-25 19:26:30,38.963700,35.243300,"2,674,766.00","28,358.00","267,476,600.00%",28358,3171.443668,nan,nan,1.060205,792,TUR


In [40]:
# import ipywidgets as widgets
# from ipywidgets import interactive
 
# items = ['All']+sorted(sorted_country_df['country'].unique().tolist())
 
def view2(y=3):
    return sorted_country_df.head(y).style.apply(highlight_col, axis = None) 
a_slider = widgets.IntSlider(min=0, max=30, step=1, value=5)

widgets.interact(view2, y= a_slider)                           

interactive(children=(IntSlider(value=5, description='y', max=30), Output()), _dom_classes=('widget-interact',…

<function __main__.view2(y=3)>

## Import Data Visulation Library
There are many Python Graphical libraries that can be used. In this case we are going to use a library called Plotly. It can be installed using Pip Install or Conda install. It can be found @ https://plotly.com/python/ 

In [43]:
import plotly.express as px

In [44]:
fig = px.scatter(sorted_country_df.head(10), x = 'country', y = 'confirmed', size = 'confirmed',
                color = 'country', hover_name = 'country', size_max = 60)

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['last_update', 'lat', 'long', 'confirmed', 'deaths', 'recovered', 'active', 'incident_rate', 'people_tested', 'people_hospitalized', 'mortality_rate', 'uid', 'iso3'] but received: country

In [ ]:
fig.show()

In [ ]:
import plotly.graph_objects as go
def plot_cases_for_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6,8]
    line_size = [4,6]
    df_list = [confirmed_df,death_df]
    fig = go.Figure()
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:,5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,5:]), axis=0)
        else:
            x_data = np.array(list(df.iloc[:,5:].columns))
            y_data = np.sum(np.asarray(df[df['country']==country].iloc[:,5:]), axis=0)
        fig.add_trace(go.Scatter(x=x_data, y= y_data, mode = 'lines+markers',
                                    name=labels[i],
                                    line = dict(color = colors[i], width = line_size[i]),
                                     connectgaps = True,
                                     text = "Total "+ str(labels[i])+ ": "+ str(y_data[-1])
                                               ))
    fig.show()

In [ ]:
# plot_cases_for_country('Ireland')
interact(plot_cases_for_country, country ='World');


## World Maps
Folium is a Python library used for visualizing geospatial data. In other words, Folium is a Python Library that enables you to import maps from around the world and allows you to represent data on them. For this excercise we will be using the Longitude(long) and Latitude(lat) to identify where our data comes from. Again you will have to install Folium using eithe Pip install or Conda install.

In [ ]:
import folium
confirmed_df=confirmed_df.dropna(subset=['long'])

confirmed_df=confirmed_df.dropna(subset=['lat'])

In [ ]:
world_map = folium.Map(location =[11,0],tiles = 'cartodbpositron', zoom_start = 2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map
                             
                             

In [ ]:
pd.set_option('display.max_rows', confirmed_df.shape[0]+1)
confirmed_df